In [1]:
import boto3
from datetime import date
import time
import uuid
from credentials import *

## Connect to DynamoDB

Initially, we connect to our local database we started as "dynamodb" container:

In [2]:
client = boto3.client('dynamodb', 
  endpoint_url='http://dynamo:8000', 
  region_name='eu-central-1',
  aws_access_key_id=aws_access_key_id,
  aws_secret_access_key=aws_secret_access_key
)
client.list_tables()

{'TableNames': ['twitter'],
 'ResponseMetadata': {'RequestId': '943cff1d-66db-43df-9c4d-a4dafd7e0b58',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 05 Jan 2024 22:54:19 GMT',
   'x-amzn-requestid': '943cff1d-66db-43df-9c4d-a4dafd7e0b58',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2569046870',
   'content-length': '26',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

## Create Table

Let's setup the table with our attributes and indexes:

In [3]:
def create_app_table():
    return client.create_table(
        TableName="twitter",
        KeySchema=[
            {"AttributeName": "PK", "KeyType": "HASH"},
            {"AttributeName": "SK", "KeyType": "RANGE"},
        ],
        AttributeDefinitions=[
            {"AttributeName": "PK", "AttributeType": "S"},
            {"AttributeName": "SK", "AttributeType": "S"},
        ],
        ProvisionedThroughput={"ReadCapacityUnits": 10, "WriteCapacityUnits": 10},
    )

In [4]:
def delete_app_table():
    try:
        return client.delete_table(TableName="twitter")
    except:
        return False

In [5]:
delete_app_table()
create_app_table()

{'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'PK',
    'AttributeType': 'S'},
   {'AttributeName': 'SK', 'AttributeType': 'S'}],
  'TableName': 'twitter',
  'KeySchema': [{'AttributeName': 'PK', 'KeyType': 'HASH'},
   {'AttributeName': 'SK', 'KeyType': 'RANGE'}],
  'TableStatus': 'ACTIVE',
  'CreationDateTime': datetime.datetime(2024, 1, 5, 22, 54, 19, 589000, tzinfo=tzlocal()),
  'ProvisionedThroughput': {'LastIncreaseDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
   'LastDecreaseDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
   'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 10,
   'WriteCapacityUnits': 10},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:ddblocal:000000000000:table/twitter',
  'DeletionProtectionEnabled': False},
 'ResponseMetadata': {'RequestId': 'da9062d1-a138-4eda-a7f9-664f23f798c8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 05 Jan 2024 22:54:19 GMT',
   'x-amzn

## Helpers

To setup some other keys later on, we create a function to extract the id from a key:

In [6]:
def parse_id_from_key(key):
    return key.split("#")[-1]

parse_id_from_key("CHAN#7")

'7'

## Users

We create and save our users:

In [7]:
uuid_entry1 = str(uuid.uuid4())
user_martin = {
    'PK': {'S': f"USERS"},
    'SK': {'S': f"USER#{uuid_entry1}"},
    'Name': {'S': 'Martin Marsal'},
    'Tweets': {'L': []},
    'Followers': {'L': []},
    'Timeline': {'L': []},
}

uuid_entry2 = str(uuid.uuid4())
user_christian = {
    'PK': {'S': f"USERS"},
    'SK': {'S': f"USER#{uuid_entry2}"},
    'Name': {'S': 'Christian Diegmann'},
    'Tweets': {'L': []},
    'Followers': {'L': []},
    'Timeline': {'L': []},
}

uuid_entry3 = str(uuid.uuid4())
user_robin = {
    'PK': {'S': f"USERS"},
    'SK': {'S': f"USER#{uuid_entry3}"},
    'Name': {'S': 'Robin Schüle'},
    'Tweets': {'L': []},
    'Followers': {'L': []},
    'Timeline': {'L': []},
}

def save_user(user):
    return client.put_item(
        TableName="twitter",
        Item=user
    )

# AP6
def find_user(primaryKey, sortKey):
    item = client.get_item(
        TableName="twitter",
        Key={
          'PK': { 'S': primaryKey },
          'SK': { 'S': sortKey }
        }
    )
    return item['Item'] if 'Item' in item else False

In [8]:
save_user(user_martin)
find_user(user_martin['PK']['S'], user_martin['SK']['S'])

{'SK': {'S': 'USER#5d020ed9-9426-4dc1-b8bf-01da78c103f3'},
 'Tweets': {'L': []},
 'Timeline': {'L': []},
 'PK': {'S': 'USERS'},
 'Followers': {'L': []},
 'Name': {'S': 'Martin Marsal'}}

In [9]:
save_user(user_christian)
find_user(user_christian['PK']['S'], user_christian['SK']['S'])

{'SK': {'S': 'USER#aa765371-b6e4-43e0-b031-32b478b068b6'},
 'Tweets': {'L': []},
 'Timeline': {'L': []},
 'PK': {'S': 'USERS'},
 'Followers': {'L': []},
 'Name': {'S': 'Christian Diegmann'}}

In [10]:
save_user(user_robin)
find_user(user_robin['PK']['S'], user_robin['SK']['S'])

{'SK': {'S': 'USER#819a5fd7-4d37-42b9-aa32-41eb851692b9'},
 'Tweets': {'L': []},
 'Timeline': {'L': []},
 'PK': {'S': 'USERS'},
 'Followers': {'L': []},
 'Name': {'S': 'Robin Schüle'}}

In [11]:
def add_to_followers(primaryKey, sortKey, follower_id):
    response = client.update_item(
        TableName='twitter',
        Key={
            'PK': {'S': primaryKey},
            'SK': {'S': sortKey}
        },
        UpdateExpression='SET Followers = list_append(if_not_exists(Followers, :empty_list), :follower)',
        ExpressionAttributeValues={
            ':empty_list': {'L': []},
            ':follower': {'L': [{'S': follower_id}]}
        },
        ReturnValues='UPDATED_NEW'
    )
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        return client.get_item(
            TableName='twitter',
            Key={
                'PK': {'S': primaryKey},
                'SK': {'S': sortKey}
            }
        )

In [12]:
add_to_followers(user_martin['PK']['S'], user_martin['SK']['S'], parse_id_from_key(user_christian['SK']['S']))

{'Item': {'SK': {'S': 'USER#5d020ed9-9426-4dc1-b8bf-01da78c103f3'},
  'Tweets': {'L': []},
  'Timeline': {'L': []},
  'PK': {'S': 'USERS'},
  'Followers': {'L': [{'S': 'aa765371-b6e4-43e0-b031-32b478b068b6'}]},
  'Name': {'S': 'Martin Marsal'}},
 'ResponseMetadata': {'RequestId': 'bda69357-4faa-4e1a-ad37-a3dd118ab392',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 05 Jan 2024 22:54:19 GMT',
   'x-amzn-requestid': 'bda69357-4faa-4e1a-ad37-a3dd118ab392',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '171955325',
   'content-length': '216',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

In [13]:
add_to_followers(user_christian['PK']['S'], user_christian['SK']['S'], parse_id_from_key(user_robin['SK']['S']))

{'Item': {'SK': {'S': 'USER#aa765371-b6e4-43e0-b031-32b478b068b6'},
  'Tweets': {'L': []},
  'Timeline': {'L': []},
  'PK': {'S': 'USERS'},
  'Followers': {'L': [{'S': '819a5fd7-4d37-42b9-aa32-41eb851692b9'}]},
  'Name': {'S': 'Christian Diegmann'}},
 'ResponseMetadata': {'RequestId': 'bf4c5a29-1e6a-48fb-9682-0700aa1ba90b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 05 Jan 2024 22:54:19 GMT',
   'x-amzn-requestid': 'bf4c5a29-1e6a-48fb-9682-0700aa1ba90b',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '574490828',
   'content-length': '221',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

In [14]:
add_to_followers(user_robin['PK']['S'], user_robin['SK']['S'], parse_id_from_key(user_martin['SK']['S']))

{'Item': {'SK': {'S': 'USER#819a5fd7-4d37-42b9-aa32-41eb851692b9'},
  'Tweets': {'L': []},
  'Timeline': {'L': []},
  'PK': {'S': 'USERS'},
  'Followers': {'L': [{'S': '5d020ed9-9426-4dc1-b8bf-01da78c103f3'}]},
  'Name': {'S': 'Robin Schüle'}},
 'ResponseMetadata': {'RequestId': '05d5cd7d-e4c9-4427-860c-1124dbcd94ea',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 05 Jan 2024 22:54:19 GMT',
   'x-amzn-requestid': '05d5cd7d-e4c9-4427-860c-1124dbcd94ea',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '3612163676',
   'content-length': '216',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

## 1st access pattern: Post a tweet


In [15]:
uuid_entry4 = str(uuid.uuid4())
tweet = {'id': {'S': f"{uuid_entry4}"}, 'text': {'S': 'Moin, moin.'}, 'likes': {'N': '0'}, 'replies': {'L': []}}
uuid_entry5 = str(uuid.uuid4())
tweet_2 = {'id': {'S': f"{uuid_entry5}"}, 'text': {'S': 'MongoDB ist super!'}, 'likes': {'N': '0'}, 'replies': {'L': []}}

def post_tweet(primaryKey, sortKey, tweet):
    client.update_item(
        TableName='twitter',
        Key={
            'PK': {'S': primaryKey},
            'SK': {'S': sortKey}
        },
        UpdateExpression='SET Tweets = list_append(if_not_exists(Tweets, :empty_list), :tweet)',
        ExpressionAttributeValues={
            ':empty_list': {'L': []},
            ':tweet': {'L': [{'M': tweet}]}
        },
        ReturnValues='UPDATED_NEW'
    )
    newMartin = find_user(user_martin['PK']['S'], user_martin['SK']['S'])
    for follower in newMartin['Followers']['L']:
        response = client.update_item(
            TableName='twitter',
            Key={
                'PK': {'S': primaryKey},
                'SK': {'S': f"USER#{follower['S']}"}
            },
            UpdateExpression='SET Timeline = list_append(if_not_exists(Timeline, :empty_list), :tweet)',
            ExpressionAttributeValues={
                ':empty_list': {'L': []},
                ':tweet': {'L': [{'M': tweet}]}
            },
            ReturnValues='UPDATED_NEW'
        )
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        return client.get_item(
            TableName='twitter',
            Key={
                'PK': {'S': primaryKey},
                'SK': {'S': sortKey}
            }
        )

In [16]:
post_tweet(user_martin['PK']['S'], user_martin['SK']['S'], tweet)

{'Item': {'SK': {'S': 'USER#5d020ed9-9426-4dc1-b8bf-01da78c103f3'},
  'Tweets': {'L': [{'M': {'id': {'S': '0da01a6b-9cc1-480f-82e1-e492363866e2'},
      'text': {'S': 'Moin, moin.'},
      'replies': {'L': []},
      'likes': {'N': '0'}}}]},
  'Timeline': {'L': []},
  'PK': {'S': 'USERS'},
  'Followers': {'L': [{'S': 'aa765371-b6e4-43e0-b031-32b478b068b6'}]},
  'Name': {'S': 'Martin Marsal'}},
 'ResponseMetadata': {'RequestId': '34bbfc2d-16f3-43ef-a4ed-63a66055d7de',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 05 Jan 2024 22:54:19 GMT',
   'x-amzn-requestid': '34bbfc2d-16f3-43ef-a4ed-63a66055d7de',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '3673976537',
   'content-length': '337',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

In [17]:
post_tweet(user_martin['PK']['S'], user_martin['SK']['S'], tweet_2)

{'Item': {'SK': {'S': 'USER#5d020ed9-9426-4dc1-b8bf-01da78c103f3'},
  'Tweets': {'L': [{'M': {'id': {'S': '0da01a6b-9cc1-480f-82e1-e492363866e2'},
      'text': {'S': 'Moin, moin.'},
      'replies': {'L': []},
      'likes': {'N': '0'}}},
    {'M': {'id': {'S': '2b82a365-97dc-4257-bb69-5a76398dbbf7'},
      'text': {'S': 'MongoDB ist super!'},
      'replies': {'L': []},
      'likes': {'N': '0'}}}]},
  'Timeline': {'L': []},
  'PK': {'S': 'USERS'},
  'Followers': {'L': [{'S': 'aa765371-b6e4-43e0-b031-32b478b068b6'}]},
  'Name': {'S': 'Martin Marsal'}},
 'ResponseMetadata': {'RequestId': '474ffb89-40f9-455e-893b-3981466296c1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 05 Jan 2024 22:54:19 GMT',
   'x-amzn-requestid': '474ffb89-40f9-455e-893b-3981466296c1',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '492534746',
   'content-length': '466',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

In [18]:
find_user(user_christian['PK']['S'], user_christian['SK']['S'])

{'SK': {'S': 'USER#aa765371-b6e4-43e0-b031-32b478b068b6'},
 'Tweets': {'L': []},
 'Timeline': {'L': [{'M': {'id': {'S': '0da01a6b-9cc1-480f-82e1-e492363866e2'},
     'text': {'S': 'Moin, moin.'},
     'replies': {'L': []},
     'likes': {'N': '0'}}},
   {'M': {'id': {'S': '2b82a365-97dc-4257-bb69-5a76398dbbf7'},
     'text': {'S': 'MongoDB ist super!'},
     'replies': {'L': []},
     'likes': {'N': '0'}}}]},
 'PK': {'S': 'USERS'},
 'Followers': {'L': [{'S': '819a5fd7-4d37-42b9-aa32-41eb851692b9'}]},
 'Name': {'S': 'Christian Diegmann'}}

In [19]:
find_user(user_robin['PK']['S'], user_robin['SK']['S'])

{'SK': {'S': 'USER#819a5fd7-4d37-42b9-aa32-41eb851692b9'},
 'Tweets': {'L': []},
 'Timeline': {'L': []},
 'PK': {'S': 'USERS'},
 'Followers': {'L': [{'S': '5d020ed9-9426-4dc1-b8bf-01da78c103f3'}]},
 'Name': {'S': 'Robin Schüle'}}

## 3rd access pattern: Edit a tweet

In [20]:
def edit_tweet(primaryKey, sortKey, tweet_id, new_text):
    
    user = find_user(primaryKey, sortKey)

    
    if user and 'Tweets' in user:
        tweets = user['Tweets']['L']

        
        tweet_index = None
        for i, tweet_item in enumerate(tweets):
            if tweet_item['M']['id']['S'] == tweet_id:
                tweet_index = i
                break

        
        if tweet_index is not None:
            tweets[tweet_index]['M']['text']['S'] = new_text

            response = client.update_item(
                TableName='twitter',
                Key={
                    'PK': {'S': primaryKey},
                    'SK': {'S': sortKey}
                },
                UpdateExpression='SET Tweets = :tweets',
                ExpressionAttributeValues={':tweets': {'L': tweets}},
                ReturnValues='UPDATED_NEW'
            )

            # Update timelines
            for follower in user['Followers']['L']:
                follower_user = find_user(user['PK']['S'], f"USER#{follower['S']}")
                if follower_user and 'Timeline' in follower_user:
                    timeline = follower_user['Timeline']['L']
                    if any(tweet['M']['id']['S'] == tweet_id for tweet in timeline):
                        # Update the edited tweet in the follower's timeline
                        for tweet_item in timeline:
                            if tweet_item['M']['id']['S'] == tweet_id:
                                tweet_item['M']['text']['S'] = new_text
                                break

                        
                        client.update_item(
                            TableName='twitter',
                            Key={
                                'PK': {'S': user['PK']['S']},
                                'SK': {'S': f"USER#{follower['S']}"}
                            },
                            UpdateExpression='SET Timeline = :timeline',
                            ExpressionAttributeValues={':timeline': {'L': timeline}},
                            ReturnValues='UPDATED_NEW'
                        )

            return response
    else:
        return False

In [21]:
old_tweet_id_to_edit = uuid_entry5  
new_tweet_text = "DynamoDB is doch besser"
edit_tweet(user_martin['PK']['S'], user_martin['SK']['S'], old_tweet_id_to_edit, new_tweet_text)

{'Attributes': {'Tweets': {'L': [{'M': {'id': {'S': '0da01a6b-9cc1-480f-82e1-e492363866e2'},
      'text': {'S': 'Moin, moin.'},
      'replies': {'L': []},
      'likes': {'N': '0'}}},
    {'M': {'id': {'S': '2b82a365-97dc-4257-bb69-5a76398dbbf7'},
      'text': {'S': 'DynamoDB is doch besser'},
      'replies': {'L': []},
      'likes': {'N': '0'}}}]}},
 'ResponseMetadata': {'RequestId': '6bd0cad4-ad9e-4ae9-b069-f3c66d54f8b6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 05 Jan 2024 22:54:20 GMT',
   'x-amzn-requestid': '6bd0cad4-ad9e-4ae9-b069-f3c66d54f8b6',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '94995368',
   'content-length': '289',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

In [22]:
find_user(user_martin['PK']['S'], user_martin['SK']['S'])

{'SK': {'S': 'USER#5d020ed9-9426-4dc1-b8bf-01da78c103f3'},
 'Tweets': {'L': [{'M': {'id': {'S': '0da01a6b-9cc1-480f-82e1-e492363866e2'},
     'text': {'S': 'Moin, moin.'},
     'replies': {'L': []},
     'likes': {'N': '0'}}},
   {'M': {'id': {'S': '2b82a365-97dc-4257-bb69-5a76398dbbf7'},
     'text': {'S': 'DynamoDB is doch besser'},
     'replies': {'L': []},
     'likes': {'N': '0'}}}]},
 'Timeline': {'L': []},
 'PK': {'S': 'USERS'},
 'Followers': {'L': [{'S': 'aa765371-b6e4-43e0-b031-32b478b068b6'}]},
 'Name': {'S': 'Martin Marsal'}}

In [23]:
find_user(user_christian['PK']['S'], user_christian['SK']['S'])

{'SK': {'S': 'USER#aa765371-b6e4-43e0-b031-32b478b068b6'},
 'Tweets': {'L': []},
 'Timeline': {'L': [{'M': {'id': {'S': '0da01a6b-9cc1-480f-82e1-e492363866e2'},
     'text': {'S': 'Moin, moin.'},
     'replies': {'L': []},
     'likes': {'N': '0'}}},
   {'M': {'id': {'S': '2b82a365-97dc-4257-bb69-5a76398dbbf7'},
     'text': {'S': 'DynamoDB is doch besser'},
     'replies': {'L': []},
     'likes': {'N': '0'}}}]},
 'PK': {'S': 'USERS'},
 'Followers': {'L': [{'S': '819a5fd7-4d37-42b9-aa32-41eb851692b9'}]},
 'Name': {'S': 'Christian Diegmann'}}

## Channel

We create our channel and functions to save, find and count users and messages in the channel:

In [24]:
channel_town_hall = {
    'PK': { 'S': "CHAN#7" },
    'SK': { 'S': "CHAN#7" },
    'Name': { 'S': 'Town Hall' },
    'Desc': { 'S': 'General News' },
    'UserCount': { 'N': "0" },
    'MessageCount': { 'N': "0" }
}

def save_channel(channel):
    return client.put_item(
        TableName="chat",
        Item=channel
    )

# AP2
def find_channel(key):
    item = client.get_item(
        TableName="chat",
        Key={
          'PK': { 'S': key },
          'SK': { 'S': key }
        }
    )
    return item['Item'] if 'Item' in item else False

# AP4
def message_count_for_channel(channel):
    return int(channel['MessageCount']['N'])

# AP7
def user_count_for_channel(channel):
    return int(channel['UserCount']['N'])

save_channel(channel_town_hall)
channel_town_hall = find_channel(channel_town_hall['PK']['S'])
channel_town_hall

ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the PutItem operation: Cannot do operations on a non-existent table

In [ ]:
message_count_for_channel(channel_town_hall)

In [ ]:
user_count_for_channel(channel_town_hall)

## User Join

When a user joins a channel, we create a new item and increment the user counter. Further, we setup queries to retreive the users for a channel and channels of a user:

In [ ]:
def join_channel(user, channel):
    # create userj record
    client.put_item(
        TableName="chat",
        Item={
            'PK': channel['PK'],
            'SK': { 'S': "USERJ#" + parse_id_from_key(user['PK']['S']) },
            'Name': user['Name'],
            'JoinedAt': { 'S': str(date.today()) },
            'GSI1PK': user['PK'],
            'GSI1SK': channel['PK'],
            'ChanName': channel['Name']
        }
        )
    
    # increment users
    client.update_item(
        TableName="chat",
        Key = {
            'PK': channel['PK'],
            'SK': channel['PK'],
        },
        ExpressionAttributeValues = {
            ':one': { 'N': '1' }
        },
        UpdateExpression = 'ADD UserCount :one', 
        ReturnValues = 'UPDATED_NEW'
      )

# AP5
def users_in_channel(key):
    item = client.query(
        TableName="chat",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :userj)',
        ExpressionAttributeValues={
            ':pk': key,
            ':userj': { 'S': 'USERJ#' }
        }
    )
    return item['Items'] if 'Items' in item else []

# AP1
def channels_for_user(key):
    item = client.query(
        TableName="chat",
        IndexName="GSI1",
        KeyConditionExpression='GSI1PK = :pk AND begins_with(GSI1SK, :chan)',
        ExpressionAttributeValues={
            ':pk': key,
            ':chan': { 'S': 'CHAN#' }
        }
    )
    return item['Items'] if 'Items' in item else []

join_channel(user_alice, channel_town_hall)
join_channel(user_bob, channel_town_hall)
users_in_channel(channel_town_hall['PK'])

In [ ]:
channels_for_user(user_alice['PK'])

In [ ]:
find_channel(channel_town_hall['PK']['S'])

## Messages

When we send a message, we create a new item and increment the messages counter:

In [ ]:
def send_message(user, channel, text):
    timestamp = int(time.time())
    message_id = uuid.uuid4()
    
    # create message
    client.put_item(
        TableName="chat",
        Item={
            'PK': channel['PK'],
            'SK': { 'S': f"MSG#{timestamp}#{message_id}" },
            'Msg': { 'S': text },
            'CreatedAt': { 'N': str(timestamp) },
            'UserName': user['Name'],
            'UserId': user['PK']
        }
        )
    
    # increment users
    client.update_item(
        TableName="chat",
        Key = {
            'PK': channel['PK'],
            'SK': channel['PK'],
        },
        ExpressionAttributeValues = {
            ':one': { 'N': '1' }
        },
        UpdateExpression = 'ADD MessageCount :one', 
        ReturnValues = 'UPDATED_NEW'
      )

# AP3
def messages_in_channel(key, limit = 50):
    item = client.query(
        TableName="chat",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :msg)',
        ExpressionAttributeValues={
            ':pk': key,
            ':msg': { 'S': 'MSG#' }
        },
        Limit = limit
    )
    return item['Items'] if 'Items' in item else []
    

send_message(user_bob, channel_town_hall, "Hey there!")
send_message(user_alice, channel_town_hall, "Hello Bob!")
send_message(user_bob, channel_town_hall, "How are you?")
messages_in_channel(channel_town_hall['PK'])

In [ ]:
find_channel(channel_town_hall['PK']['S'])